In [1]:
from selenium.webdriver.common.by import By
import random
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image, ImageDraw, ImageFont
import torchvision.transforms as standard_transforms
import torch

def append_hanzi(raw_img, character):
    '''选取五个汉字中的一个，将其放在图片右侧'''
    out_img = Image.new('L', (360, 160), (0))
    draw = ImageDraw.Draw(out_img)
    draw.text((320, 0), character, fill=(255), font=ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\ttc\msyh.ttc", 28, encoding="unic"))
    draw.text((320, 40), character, fill=(255), font=ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\ttc\STXINGKA.TTF", 40, encoding="unic"))
    draw.text((320, 80), character, fill=(255), font=ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\ttc\STZHONGS.TTF", 30, encoding="unic"))
    draw.text((320, 120), character, fill=(255), font=ImageFont.truetype(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\ttc\SIMLI.TTF", 30, encoding="unic"))
    out_img.paste(raw_img, (0, 0), mask=raw_img)
    return out_img

def preprocess(image, character):
    # 灰度化
    image = image.convert('L')
    # 向图片右侧增扩汉字图像
    image = append_hanzi(image, character)
    # 暂存图像
    image.save(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\junk\preprocessed.jpg")
    return image

def get_pos(model, character, img_path):
    # PIL Image 转 Tensor
    transform = standard_transforms.ToTensor()
    # 将图片转化为 tensor
    image = Image.open(img_path)
    # 图片预处理
    image = preprocess(image, character)
    image_tensor = transform(image)
    image_tensor = torch.unsqueeze(image_tensor, 0)
    pos = model(image_tensor.to('cuda')).cpu()
    if pos[0][0]<0:
        pos[0][0] = 0
    if pos[0][0]>=360:
        pos[0][0] = 359
    if pos[0][1]<0:
        pos[0][2] = 0
    if pos[0][1]>=160:
        pos[0][1] = 159
    return [pos[0][0].item(), pos[0][1].item()]

def click(driver):
    # 加载模型
    model = torch.load(r"D:\vscode_workspace\Python\ZhongYi\chaoxin\CNN_Network\CNN.pth")
    model.eval()
    # 获取验证码的图片及其位置和大小
    captcha_img = driver.find_element(By.CLASS_NAME, 'cx_imgBg')
    captcha_x_pos = captcha_img.location.get('x')
    captcha_y_pos = captcha_img.location.get('y')
    captcha_width = captcha_img.size.get('width')
    captcha_height = captcha_img.size.get('height')
    img_path = 'D:/vscode_workspace/Python/ZhongYi/chaoxin/junk/VerifyCode'+str(random.randint(0, 1000))+'.jpg'
    captcha_img.screenshot(img_path)
    # 获取三个待点击的汉字
    tmp_bs = BeautifulSoup(driver.page_source, 'html.parser')
    target_text = tmp_bs.find("span",{"class":{"cx_tip_text cx_fallback_tip"}}).b.text.replace(' ','').replace('\"','')
    # 获取汉字位置，然后点击
    for character in target_text:
        pos_list = get_pos(model, character, img_path)
        print("汉字", character, "的坐标为", pos_list)
        character_x_pos = pos_list[0]
        character_y_pos = pos_list[1]
        move_x = captcha_x_pos+captcha_width*(character_x_pos/320)
        move_y = captcha_y_pos+captcha_height*(character_y_pos/160)
        ActionChains(driver).move_by_offset(move_x, move_y).click().perform()
        ActionChains(driver).move_by_offset(-move_x, -move_y).perform()

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

if 1:
    chaoxin_href = r"http://fx.sjtulib.superlib.net/verifyPage?key=fea955955053474970e8ad158286b0b8&index=1&enc=f252c4d2257f2d07afb38f9a668b7b11&backUrl=http%3A%2F%2Ffx.sjtulib.superlib.net%2Fs%3Fsw%3D%25E4%25B8%25AD%25E5%258C%25BB%25E8%258D%25AF%26size%3D15%26isort%3D%26x%3D788_1071%26pages%3D70%26version%3Dv2"
    myoption = webdriver.ChromeOptions()
    myoption.add_experimental_option('detach', True)
    myoption.add_argument('-ignore-certificate-errors')
    myoption.add_argument('-ignore -ssl-errors')
    myoption.add_experimental_option("excludeSwitches", ['enable-automation', 'enable-logging'])
    driver = webdriver.Chrome(r"D:\vscode_workspace\Python\ZhongYi\chromedriver_win64\chromedriver.exe", options=myoption)  # 创建浏览器实例
    driver.get(chaoxin_href)
    driver.maximize_window()
    # 创建 WebDriverWait 对象，最大等待时间为 60 秒
    wait = WebDriverWait(driver, 60)
    # 等待页面加载
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "cx_imgBg")))
    # 点击验证码汉字
    click(driver)
    # 等待程序结束
    print("程序运行结束，请输入任意字符以返回")
    input()


C:\Users\15692\AppData\Local\Temp\ipykernel_10672\1916015847.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"D:\vscode_workspace\Python\ZhongYi\chromedriver_win64\chromedriver.exe", options=myoption)  # 创建浏览器实例
d:\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py:352: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  warnings.warn(


汉字 等 的坐标为 [181.7921142578125, 132.34080505371094]
汉字 包 的坐标为 [196.50592041015625, 140.1470947265625]
汉字 业 的坐标为 [189.52716064453125, 131.15061950683594]
程序运行结束，请输入任意字符以返回
